In [7]:
import boto3
from botocore.exceptions import ClientError

In [4]:
region = "us-west-1"
bucket_name = "chatbot-ld5ehom"

s3 = boto3.client("s3", region_name=region)

def create_bucket(bucket_name, region):
    try:
        location = {"LocationConstraint": region}
        s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration=location
        )
        print("✅ Bucket created:", bucket_name)
    except ClientError as e:
        print("❌ Bucket creation error:", e)

create_bucket("rag-example-2", region)

✅ Bucket created: rag-example-2


In [5]:
bucket_name = "chatbot-ld5ehom"
def check_bucket_connection(bucket_name):
    try: 
        response = s3.list_objects_v2(Bucket=bucket_name)
        print("Success")
        contents = response.get("Contents", [])

        if contents:
            for obj in contents:
                print(obj["Key"])
        else: 
            print("Bucket is Empty")
    except ClientError as e:
        print(e)
check_bucket_connection(bucket_name)

Success
Bucket is Empty


In [8]:
from notion_client import Client
import re

In [11]:
notion = Client(auth="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")

search_result = notion.search(
    filter={"property":"object","value":"page"},
    sort={"direction":"descending", "timestamp":"last_edited_time"}
)

result = search_result["results"][1]
title = "None"

props = result.get("properties", {})

if "title" in props:
    title_list = props["title"].get("title",[])
    if title_list:
        title = title_list[0].get("plain_text", "None")

print(result["id"])
print(title)


23695a9a-7e1b-804e-bf15-cd78254cfa5c
Setup


In [12]:
search_result["results"][1]

{'object': 'page',
 'id': '23695a9a-7e1b-804e-bf15-cd78254cfa5c',
 'created_time': '2025-07-20T05:03:00.000Z',
 'last_edited_time': '2025-07-20T05:13:00.000Z',
 'created_by': {'object': 'user',
  'id': '199d872b-594c-812b-ac8b-00024f879c96'},
 'last_edited_by': {'object': 'user',
  'id': '199d872b-594c-812b-ac8b-00024f879c96'},
 'cover': None,
 'icon': None,
 'parent': {'type': 'workspace', 'workspace': True},
 'archived': False,
 'in_trash': False,
 'properties': {'title': {'id': 'title',
   'type': 'title',
   'title': [{'type': 'text',
     'text': {'content': 'Setup', 'link': None},
     'annotations': {'bold': False,
      'italic': False,
      'strikethrough': False,
      'underline': False,
      'code': False,
      'color': 'default'},
     'plain_text': 'Setup',
     'href': None}]}},
 'url': 'https://www.notion.so/Setup-23695a9a7e1b804ebf15cd78254cfa5c',
 'public_url': None}

In [ ]:
def extract_sort_key(page):
    title_obj = page["properties"]["title"]["title"]
    if not title_obj:
        return float("inf")
    title_text = title_obj[0]["plain_text"]
    match = re.match(r"(\d+)", title_text)
    return int(match.group(1)) if match else float("inf")

for i, result in enumerate(sorted_results, start=1):
    title="None"
    props = result.get("properties", {})
    if "title" in props: 
        title_list = props["title"].get("title", [])
        if title_list: 
            title = title_list[0].get("plain_text", "None")

    print(result["id"])
    print(title)


23695a9a-7e1b-808e-9ef3-fe84a04fcaaa
HR
23695a9a-7e1b-804e-bf15-cd78254cfa5c
Setup
23695a9a-7e1b-8029-af00-cdf87b225321
설정
23695a9a-7e1b-8085-a712-c260a41eacd0
인사
